# Data Clearning & Preprocessing

In [1]:
# Import dependencies
import pandas as pd
import os

In [2]:
# Set data directory path
DATA_PATH = './data/'

## Step 1: Extracting data

Extract the dataset in `./data` directory

(This is a one time step, it need not be performed everytime)

In [3]:
!ls

Anonymous_BetchCSE_Student_Data.zip     data-cleaning-preprocessing-Copy1.ipynb
README.md                               data-cleaning-preprocessing.ipynb
data                                    plots


In [4]:
# !mkdir data
# !unzip Anonymous_BetchCSE_Student_Data.zip -d data/

## Step 2: Merging Data File

The data is splitted into 7 different Batches[1-7].

Preprocess them, Merge them together to use further.

In [5]:
# Get files listed in ./data/
files = sorted(os.listdir(DATA_PATH))
files

['.DS_Store',
 'Batch 1.xls',
 'Batch 2.xls',
 'Batch 3.xls',
 'Batch 4.xls',
 'Batch 5.xls',
 'Batch 6.xls',
 'Batch 7.xls',
 'preprocessed']

In [6]:
df = pd.read_excel(DATA_PATH + 'Batch 1.xls')
df.head()

Indraprastha Institute of Information Technology, Delhi Unnamed: 1  \
0                                                NaN             NaN   
1                                                NaN             NaN   
2                                                NaN             NaN   
3                                                SN.         Roll No   
4                                                  1       Student 1   

          Unnamed: 2   Unnamed: 3        Unnamed: 4 Unnamed: 5 Unnamed: 6  \
0                NaN          NaN               NaN        NaN        NaN   
1                NaN          NaN               NaN        NaN        NaN   
2                NaN          NaN               NaN        NaN        NaN   
3  Batch / Term Code  Course Code            Course     Credit      Grade   
4         Semester 5       CSE535  Mobile Computing          4          B   

  Unnamed: 7 Unnamed: 8  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        SPI        CPI  
4       9.17       9.23

### Step 2.a: Preprocess Data

The above observations tells us that the data is not properly formatted for direct use.

Following steps are needed to rectify this:

* Set the 3rd row as column headers
* Drop the first 4 rows
* Reset the indexing of the dataframe
* Drop extra columns if any

In [7]:
def preprocess_file(df):
    df.columns = df.iloc[3]
    df.drop([0,1,2,3], inplace=True)
    df.reset_index(inplace=True)
    df.drop('index', axis=1, inplace=True)

### Step 2.b: Merge Data
The Batch files has the same student roll numbers for different students as they belong to a different batch.

Before merging we need to add a column to identify **`Student 'x'`** is from which **`Batch 'y'`**.

To do this we add a new column **`'Student-Batch'`** with values **`'Student x Batch y'`** & column **`'Batch'`** with values **`'Batch y'`**.

In [8]:
# Remove the files which are hidden or not excel sheets
for i, file in enumerate(files):
    if file.split('.')[-1] != 'xls':
        print('Found a non-excel file, removing: ', file)
        files.pop(i)
        
df = None
for i, file in enumerate(files, start=1):
    # get the batch's data frame
    temp_df = pd.read_excel(DATA_PATH + file)
    
    # preprocess the batch dataframe to get required format
    preprocess_file(temp_df)
    
    # Add Student-Batch & Batch column
    temp_df['Batch'] = pd.Series(f'Batch {i}', index=temp_df.index)
    temp_df["Student-Batch"] = temp_df['Roll No'] + " " +temp_df['Batch']
    if df is not None:
        df = df.append(temp_df, ignore_index=True)
    else:
        df = temp_df    

Found a non-excel file, removing:  .DS_Store
Found a non-excel file, removing:  preprocessed


In [9]:
df.tail()

3       SN.      Roll No Batch / Term Code Course Code  \
28680  5520  Student 131        Semester 3      ECO301   
28681  5521  Student 131        Semester 7      CSE643   
28682  5522  Student 131        Semester 8      CSE341   
28683  5523  Student 131        Semester 5   CSE398A-1   
28684  5524  Student 131     Summer Term 2      ENT403   

3                                 Course Credit Grade   SPI   CPI    Batch  \
28680  Introduction to Economic Analysis      4    A-   8.6  9.13  Batch 7   
28681            Artificial Intelligence      4     A   9.5  9.33  Batch 7   
28682                         Biometrics      4     A   9.6  9.37  Batch 7   
28683                  Independent Study      4     C   8.6  8.92  Batch 7   
28684                 Starting a Venture      4     A  10.0   9.0  Batch 7   

3            Student-Batch  
28680  Student 131 Batch 7  
28681  Student 131 Batch 7  
28682  Student 131 Batch 7  
28683  Student 131 Batch 7  
28684  Student 131 Batch 7

In [10]:
# See the number of rows, columns in the new dataframe
df.shape

(28685, 11)

## Step 3: Export Data

Save the new dataframe created in a new excel to avoid the preprocessing step in futue.

In [11]:
final_file = 'MergedData.xlsx'
NEW_DATA_PATH = DATA_PATH + 'preprocessed/'
os.mkdir(NEW_DATA_PATH)

In [12]:
df.to_excel(NEW_DATA_PATH + final_file)

## Step 4: Load Merged Data

In future we can directly run below step to load the merged data.

In [13]:
df = pd.read_excel(NEW_DATA_PATH + final_file)

In [14]:
df.head()

SN.    Roll No Batch / Term Code Course Code  \
0    1  Student 1        Semester 5      CSE535   
1    2  Student 1        Semester 3    COM201-1   
2    3  Student 1        Semester 6      CSE546   
3    4  Student 1        Semester 5      CSE300   
4    5  Student 1        Semester 4      CSE222   

                              Course  Credit Grade   SPI   CPI    Batch  \
0                   Mobile Computing       4     B  9.17  9.23  Batch 1   
1                   Critical Reading       2    A-  9.67  9.27  Batch 1   
2               Applied Cryptography       4     A  8.80  9.16  Batch 1   
3              Software Engineering        4    A-  9.17  9.23  Batch 1   
4  Analysis and Design of Algorithms       4     B  9.20  9.25  Batch 1   

       Student-Batch  
0  Student 1 Batch 1  
1  Student 1 Batch 1  
2  Student 1 Batch 1  
3  Student 1 Batch 1  
4  Student 1 Batch 1

## Step 5: Exploring Dataset

This step is crucial to further preprocess and clean our dataset as per our requirements.

In [15]:
# Get rows which has null value in any of its columns
df[df.isnull().any(axis=1)]

SN.      Roll No Batch / Term Code Course Code  \
8         9    Student 1     Summer Term 2      MSC482   
56       57    Student 2     Summer Term 3      MSC492   
81       82    Student 2     Summer Term 3      MSC482   
126     127    Student 5     Summer Term 3      MSC481   
241     242    Student 8     Summer Term 3      MSC482   
273     274    Student 9     Summer Term 2      MSC492   
285     286    Student 9     Summer Term 3      MSC482   
329     330   Student 10     Summer Term 3      MSC492   
339     340   Student 10     Summer Term 3      MSC482   
458     459   Student 14     Summer Term 3      MSC481   
469     470   Student 14     Summer Term 3      MSC492   
505     506   Student 15     Summer Term 3      MSC481   
522     523   Student 15     Summer Term 3      MSC492   
568     569   Student 17     Summer Term 3      MSC492   
595     596   Student 17     Summer Term 3      MSC482   
628     629   Student 18     Summer Term 2      MSC482   
650     651   Student 19     Summer Term 3      MSC482   
670     671   Student 19     Summer Term 3      MSC492   
697     698   Student 20     Summer Term 3      MSC492   
729     730   Student 20     Summer Term 2      MSC482   
814     815   Student 23     Summer Term 2      MSC491   
820     821   Student 23     Summer Term 3      MSC482   
883     884   Student 24     Summer Term 3      MSC482   
930     931   Student 25     Summer Term 3      MSC492   
943     944   Student 27        Semester 2      CSE112   
989     990   Student 28     Summer Term 3      MSC482   
1013   1014   Student 29     Summer Term 3      MSC482   
1104   1105   Student 31     Summer Term 3      MSC482   
1107   1108   Student 31     Summer Term 3      MSC492   
1123   1124   Student 32     Summer Term 3      MSC482   
...     ...          ...               ...         ...   
27588  4428  Student 106        Semester 9      CSE511   
27589  4429  Student 106        Semester 9      SOC206   
27590  4430  Student 106        Semester 9      SSH121   
27594  4434  Student 106     Summer Term 2    CSE999A2   
27599  4439  Student 106     Summer Term 4     CSE999C   
27633  4473  Student 106        Semester 9      CSE333   
27642  4482  Student 107     Summer Term 2    CSE999A2   
27775  4615  Student 110        Semester 9      CSE300   
27812  4652  Student 110     Summer Term 4     CSE999C   
27813  4653  Student 110     Summer Term 4     SSH999B   
27814  4654  Student 110        Semester 9      ECO201   
27815  4655  Student 110        Semester 9      CSE546   
28295  5135  Student 122     Summer Term 2     CSE999B   
28301  5141  Student 122        Semester 9     CSE5IMC   
28302  5142  Student 122        Semester 9      DES5xx   
28314  5154  Student 122        Semester 9      CSE640   
28315  5155  Student 122        Semester 9      SSH221   
28316  5156  Student 122        Semester 9      CSE300   
28329  5169  Student 123     Summer Term 2    CSE999A2   
28441  5281  Student 125     Summer Term 3      MSC200   
28442  5282  Student 125     Summer Term 4     CSE999C   
28448  5288  Student 125     Summer Term 2    CSE999A2   
28548  5388  Student 127     Summer Term 2     CSE999B   
28630  5470  Student 130        Semester 9      CSE506   
28637  5477  Student 130     Summer Term 4     SSH999B   
28638  5478  Student 130     Summer Term 4     CSE999C   
28641  5481  Student 130        Semester 9      ECO331   
28642  5482  Student 130        Semester 9      CSE640   
28648  5488  Student 130        Semester 9      CSE556   
28649  5489  Student 130        Semester 9      CSE499   

                                      Course  Credit Grade   SPI   CPI  \
8                                Self Growth       2     S   NaN   NaN   
56                            Community Work       2     S   NaN   NaN   
81                               Self Growth       2     S   NaN   NaN   
126                              Self Growth       1     S   NaN   NaN   
241                              Self Gro

The above code shows us that there are missing values in a row if a student had failed `F` or did a course in `Summer Term`.

For the purpose of our project we do not want such data points and hence we'll remove any rows which has NaN values.

In [16]:
df = df[df.notnull().all(axis=1)]
df.reset_index(inplace=True)
df

index   SN.      Roll No Batch / Term Code Course Code  \
0          0     1    Student 1        Semester 5      CSE535   
1          1     2    Student 1        Semester 3    COM201-1   
2          2     3    Student 1        Semester 6      CSE546   
3          3     4    Student 1        Semester 5      CSE300   
4          4     5    Student 1        Semester 4      CSE222   
5          5     6    Student 1        Semester 5      CSE505   
6          6     7    Student 1        Semester 2      CSE112   
7          7     8    Student 1        Semester 6      CSE645   
8          9    10    Student 1        Semester 4      CSE232   
9         10    11    Student 1        Semester 6     CSE342*   
10        11    12    Student 1        Semester 1      CSE101   
11        12    13    Student 1        Semester 8      ECO301   
12        13    14    Student 1        Semester 4      ESC202   
13        14    15    Student 1        Semester 7      CSE343   
14        15    16    Student 1        Semester 7      CSE399   
15        16    17    Student 1        Semester 7      CSE499   
16        17    18    Student 1        Semester 3      CSE231   
17        18    19    Student 1        Semester 8      CSE499   
18        19    20    Student 1        Semester 3      CSE201   
19        20    21    Student 1        Semester 8     HSS204*   
20        21    22    Student 1        Semester 8      CSE606   
21        22    23    Student 1        Semester 1      CSE121   
22        23    24    Student 1        Semester 2   CSE131-1*   
23        24    25    Student 1        Semester 5      MTH101   
24        25    26    Student 1        Semester 2     CSE122*   
25        26    27    Student 1        Semester 4     HSS201*   
26        27    28    Student 1        Semester 7     CSE694C   
27        28    29    Student 1        Semester 8      ENV301   
28        29    30    Student 1        Semester 3      MTH201   
29        30    31    Student 1        Semester 2      CSE102   
...      ...   ...          ...               ...         ...   
28160  28655  5495  Student 131        Semester 6      CSE508   
28161  28656  5496  Student 131        Semester 4      CSE322   
28162  28657  5497  Student 131        Semester 4     ESC205A   
28163  28658  5498  Student 131        Semester 4      CSE222   
28164  28659  5499  Student 131        Semester 5      CSE535   
28165  28660  5500  Student 131        Semester 7     ESC999A   
28166  28661  5501  Student 131        Semester 4      CSE232   
28167  28662  5502  Student 131        Semester 3      CSE121   
28168  28663  5503  Student 131        Semester 5      CSE343   
28169  28664  5504  Student 131        Semester 7      DES513   
28170  28665  5505  Student 131        Semester 8      CSE342   
28171  28666  5506  Student 131        Semester 3      CSE231   
28172  28667  5507  Student 131        Semester 3      ECE250   
28173  28668  5508  Student 131        Semester 4     COM301A   
28174  28669  5509  Student 131        Semester 5      CSE555   
28175  28670  5510  Student 131        Semester 3      CSE201   
28176  28671  5511  Student 131        Semester 6      CSE499   
28177  28672  5512  Student 131        Semester 8      CSE609   
28178  28673  5513  Student 131        Semester 7     HSS204*   
28179  28674  5514  Student 131        Semester 6      FIN401   
28180  28675  5515  Student 131        Semester 5      CSE345   
28181  28676  5516  Student 131        Semester 4    CSE999B3   
28182  28677  5517  Student 131        Semester 4    CSE999B2   
28183  28678  5518  Student 131        Semester 6      CSE565   
28184  28679  5519  Student 131        Semester 7      CSE499   
28185  28680  5520  Student 131        Semester 3      ECO301   
28186  28681  5521  Student 131        Semester 7      CSE643   
28187  28682  5522  Student 131        Semester 8      CSE341   
28188  28683  5523  Student 131        Semester 5   CSE398A-1   
28189  28684  5524  Student 131     Summer

In [17]:
df.drop(['index', 'SN.'], axis=1, inplace=True)
df.head()

Roll No Batch / Term Code Course Code                             Course  \
0  Student 1        Semester 5      CSE535                   Mobile Computing   
1  Student 1        Semester 3    COM201-1                   Critical Reading   
2  Student 1        Semester 6      CSE546               Applied Cryptography   
3  Student 1        Semester 5      CSE300              Software Engineering    
4  Student 1        Semester 4      CSE222  Analysis and Design of Algorithms   

   Credit Grade   SPI   CPI    Batch      Student-Batch  
0       4     B  9.17  9.23  Batch 1  Student 1 Batch 1  
1       2    A-  9.67  9.27  Batch 1  Student 1 Batch 1  
2       4     A  8.80  9.16  Batch 1  Student 1 Batch 1  
3       4    A-  9.17  9.23  Batch 1  Student 1 Batch 1  
4       4     B  9.20  9.25  Batch 1  Student 1 Batch 1

Get the stats of specific colummns in the dataframe such as printing unique values or their count to analyse and rectify further.

In [18]:
print(f'Columns: {df.columns}\n Shape: {df.shape}')

Columns: Index(['Roll No', 'Batch / Term Code', 'Course Code', 'Course', 'Credit',
       'Grade', 'SPI', 'CPI', 'Batch', 'Student-Batch'],
      dtype='object')
 Shape: (28190, 10)


In [19]:
def print_table_stats(df):
    # Print the unique values in specified columns
    for col in ['Batch / Term Code', 'Credit', 'Grade']:
        print(f"Column name: '{col}'")
        print(df[col].unique())
        print()

    # Print the count of unique values in specified columns
    for col in ['Course Code', 'Course']:
        print(f"Number of unique '{col}' = {len(df[col].unique())}")
        print()

In [20]:
print_table_stats(df)

Column name: 'Batch / Term Code'
['Semester 5' 'Semester 3' 'Semester 6' 'Semester 4' 'Semester 2'
 'Semester 1' 'Semester 8' 'Semester 7' 'Summer Term 1' 'Semester 9'
 'Summer Term 4' 'Summer Term 3' 'Summer Term 2' 'Semester 10'
 'Semester 12' 'Semester 11' 'Summer Term 5' 'Semester 14' 'Semester 13']

Column name: 'Credit'
[ 4  2  8  6 12  1  3]

Column name: 'Grade'
['B' 'A-' 'A' 'B-' 'A+' 'C' 'D' 'W' 'F' 'C-' 'S' 'I' 'X']

Number of unique 'Course Code' = 309

Number of unique 'Course' = 251



In [21]:
df.loc[df['Grade'].isin(['S', 'W', 'I', 'X'])]

Roll No Batch / Term Code Course Code  \
53       Student 2        Semester 7     CSE694C   
227      Student 8        Semester 7      PHY301   
326     Student 10        Semester 6     CSE342*   
349     Student 11        Semester 9      CSE999   
517     Student 16     Summer Term 4      ECO999   
585     Student 17        Semester 6     CSE342*   
635     Student 19        Semester 6     CSE342*   
712     Student 21        Semester 7     CSE691A   
838     Student 24        Semester 6     CSE342*   
1053    Student 31        Semester 6     CSE342*   
1158    Student 33        Semester 6     CSE342*   
1192    Student 34        Semester 6     CSE342*   
1408    Student 40        Semester 7      CSE507   
1516    Student 44        Semester 8      CSE539   
1597    Student 46        Semester 7      CSE507   
2217     Student 3        Semester 5      CSE500   
2412    Student 12        Semester 4     HSS204*   
2489    Student 14        Semester 6      CSE537   
2581    Student 16        Semester 6      CSE537   
2616    Student 19        Semester 8     CSE694F   
2752    Student 23        Semester 6     CSE342*   
2916    Student 28        Semester 8      CSE999   
2917    Student 29        Semester 6     CSE342*   
3105    Student 33        Semester 6     CSE342*   
3220    Student 36        Semester 6     CSE342*   
3231    Student 37        Semester 5      CSE521   
3809    Student 53        Semester 8     HSS999*   
4090    Student 64        Semester 9      CSE609   
4533    Student 10        Semester 7      CSE632   
4865    Student 19        Semester 7      CSE640   
...            ...               ...         ...   
27870  Student 123        Semester 7     ESC999A   
27889  Student 124        Semester 8     CSE999A   
27899  Student 124        Semester 8      CSE609   
27900  Student 124        Semester 8    CSE999A2   
27902  Student 124        Semester 6     CSE999A   
27915  Student 124        Semester 7     CSE999A   
27918  Student 124        Semester 7      CSE343   
27936  Student 125        Semester 8    CSE999A2   
27943  Student 125        Semester 6      CSE507   
27965  Student 125        Semester 7     CSE999B   
27970  Student 125        Semester 8     CSE999A   
27978  Student 126        Semester 5      CSE343   
27982  Student 126        Semester 6      CSE561   
27985  Student 126     Summer Term 4     MGT999A   
27990  Student 126        Semester 7     ESC999A   
27992  Student 126     Summer Term 2    CSE999A2   
27993  Student 126        Semester 7     CSE999A   
28000  Student 126        Semester 8     CSE999A   
28030  Student 127        Semester 8     CSE999A   
28042  Student 127        Semester 8    CSE999A2   
28063  Student 127        Semester 5     BIO601*   
28064  Student 128        Semester 8     CSE999A   
28075  Student 128        Semester 6     CSE999A   
28080  Student 128        Semester 7      CSE343   
28112  Student 129        Semester 5      CSE343   
28123  Student 129        Semester 3     HSS101*   
28140  Student 130        Semester 7      CSE643   
28165  Student 131        Semester 7     ESC999A   
28181  Student 131        Semester 4    CSE999B3   
28182  Student 131        Semester 4    CSE999B2   

                                                  Course  Credit Grade    SPI  \
53           Privacy and Security in Online Social Media       2     W   7.44   
227                                              Physics       4     W   7.57   
326                                  Pattern Recognition       4     W   8.00   
349                               Distance Course in CSE       4     S   7.60   
517                         Distance Course in Economics       4     S   6.00   
585                                  Pattern Recognition       4     W   8.50   
635                                  Pattern Recognition       4     W   8.25   
712    Application Development for Web2.0 Social Medi...       2     W   5.43   
838                                  Pattern Recognition 

This tells us that some courses were withdraw 'W' by some students, This is something which we don't want future students to get recommended hence we'll remove them.

Also, there are some anomalies where SPI and CPI were given even in the Summer Term, hence we'll remove them as well.

In [22]:
summer_terms = [f'Summer Term {i}' for i in range(1, 6)]
df = df.loc[~df['Batch / Term Code'].isin(summer_terms)]
df = df.loc[~df['Grade'].isin(['W', 'I', 'X'])]

In [23]:
df.loc[df['Grade'].isin(['S', 'W', 'I', 'X'])]

Roll No Batch / Term Code Course Code                  Course  \
349     Student 11        Semester 9      CSE999  Distance Course in CSE   
2916    Student 28        Semester 8      CSE999  Distance Course in CSE   
3809    Student 53        Semester 8     HSS999*  Distance Course in HSS   
7902     Student 2        Semester 8     CSE999A  Distance Course in CSE   
8014     Student 6        Semester 8     CSE999A  Distance Course in CSE   
8260    Student 12        Semester 8     CSE999A  Distance Course in CSE   
8428    Student 16        Semester 8     CSE999A  Distance Course in CSE   
8458    Student 17       Semester 10    CSE999B3  Distance Course in CSE   
8465    Student 17       Semester 14     CSE999A  Distance Course in CSE   
8480    Student 17       Semester 14     ECE999A  Distance Course in ECE   
8688    Student 22        Semester 8     CSE999A  Distance Course in CSE   
8723    Student 23        Semester 8     CSE999A  Distance Course in CSE   
8896    Student 28        Semester 8     CSE999A  Distance Course in CSE   
8971    Student 30        Semester 8     CSE999A  Distance Course in CSE   
9054    Student 32        Semester 8     CSE999A  Distance Course in CSE   
9081    Student 33        Semester 8     CSE999A  Distance Course in CSE   
9126    Student 34        Semester 8     CSE999A  Distance Course in CSE   
9298    Student 38        Semester 8     CSE999A  Distance Course in CSE   
9418    Student 41       Semester 10    CSE999B2  Distance Course in CSE   
9477    Student 43        Semester 8     CSE999A  Distance Course in CSE   
9626   Student 119        Semester 8     CSE999A  Distance Course in CSE   
9748    Student 47        Semester 8     CSE999A  Distance Course in CSE   
9774    Student 48        Semester 8     CSE999A  Distance Course in CSE   
9910    Student 51        Semester 8     CSE999A  Distance Course in CSE   
9936    Student 52        Semester 8     CSE999A  Distance Course in CSE   
10062   Student 55        Semester 8     CSE999A  Distance Course in CSE   
10087   Student 56        Semester 8     CSE999A  Distance Course in CSE   
10153   Student 58        Semester 8     CSE999A  Distance Course in CSE   
10198   Student 59        Semester 8     CSE999A  Distance Course in CSE   
10525   Student 67        Semester 8     CSE999A  Distance Course in CSE   
...            ...               ...         ...                     ...   
27548  Student 115        Semester 8    CSE999A2  Distance Course in CSE   
27575  Student 116        Semester 8     CSE999A  Distance Course in CSE   
27581  Student 116        Semester 8    CSE999A2  Distance Course in CSE   
27626  Student 117        Semester 7     CSE999B  Distance Course in CSE   
27627  Student 117        Semester 7     ESC999A  Distance Course in ESC   
27632  Student 117        Semester 8    CSE999A2  Distance Course in CSE   
27640  Student 117        Semester 5     CSE999B  Distance Course in CSE   
27657  Student 117        Semester 6     CSE999A  Distance Course in CSE   
27708  Student 119        Semester 8    CSE999A2  Distance Course in CSE   
27729  Student 119        Semester 8     CSE999A  Distance Course in CSE   
27772  Student 121        Semester 7     CSE999A  Distance Course in CSE   
27773  Student 121        Semester 6     CSE999C  Distance Course in CSE   
27779  Student 121        Semester 6     CSE999A  Distance Course in CSE   
27834  Student 122        Semester 4    CSE999B2  Distance Course in CSE   
27850  Student 123        Semester 8     CSE999A  Distance Course in CSE   
27870  Student 123        Semester 7     ESC999A  Distance Course in ESC   
27889  Student 124        Semester 8     CSE999A  Distance Course in CSE   
27900  Student 124        Semester 8    CSE999A2  Distance Course in CSE   
27902  Student 124        Semester 6     CSE999A  Distance Course in CSE   
27915  Student 124        Semester 7     CSE999A  Distance Course in CSE   
27936  Student 125        Semester 8    CSE999A2  Distance 

In [24]:
print_table_stats(df)

Column name: 'Batch / Term Code'
['Semester 5' 'Semester 3' 'Semester 6' 'Semester 4' 'Semester 2'
 'Semester 1' 'Semester 8' 'Semester 7' 'Semester 9' 'Semester 10'
 'Semester 12' 'Semester 11' 'Semester 14' 'Semester 13']

Column name: 'Credit'
[ 4  2  8  6 12  1  3]

Column name: 'Grade'
['B' 'A-' 'A' 'B-' 'A+' 'C' 'D' 'F' 'C-' 'S']

Number of unique 'Course Code' = 293

Number of unique 'Course' = 242



There are more than 8 semesters for B.Tech students in the dataset.

This is because some students went for M.Tech or Ph.D. programmes in continuation after their B.Tech.

We do not want such data points in our dataset for recommendations for B.Tech students, hence we'll remove them as well.

In [25]:
keep_semesters = [f'Semester {i}' for i in range(1, 9)]
df = df.loc[df['Batch / Term Code'].isin(keep_semesters)]
print_table_stats(df)

Column name: 'Batch / Term Code'
['Semester 5' 'Semester 3' 'Semester 6' 'Semester 4' 'Semester 2'
 'Semester 1' 'Semester 8' 'Semester 7']

Column name: 'Credit'
[ 4  2  8  6 12  1  3]

Column name: 'Grade'
['B' 'A-' 'A' 'B-' 'A+' 'C' 'D' 'F' 'C-' 'S']

Number of unique 'Course Code' = 285

Number of unique 'Course' = 235



In [26]:
df.head()

Roll No Batch / Term Code Course Code                             Course  \
0  Student 1        Semester 5      CSE535                   Mobile Computing   
1  Student 1        Semester 3    COM201-1                   Critical Reading   
2  Student 1        Semester 6      CSE546               Applied Cryptography   
3  Student 1        Semester 5      CSE300              Software Engineering    
4  Student 1        Semester 4      CSE222  Analysis and Design of Algorithms   

   Credit Grade   SPI   CPI    Batch      Student-Batch  
0       4     B  9.17  9.23  Batch 1  Student 1 Batch 1  
1       2    A-  9.67  9.27  Batch 1  Student 1 Batch 1  
2       4     A  8.80  9.16  Batch 1  Student 1 Batch 1  
3       4    A-  9.17  9.23  Batch 1  Student 1 Batch 1  
4       4     B  9.20  9.25  Batch 1  Student 1 Batch 1

## Step 6: Save Preprocessed Data

Save the preprocessed data in a complete file merged together as well as Batch-wise preprocessed data for further dataset analysis.

In [27]:
processed_file_name = 'Preprocessed_'+ final_file
df.to_excel(NEW_DATA_PATH + processed_file_name)

In [28]:
grouped = df.groupby('Batch')
writer = pd.ExcelWriter(NEW_DATA_PATH + 'Preprocessed_Batches.xlsx')
for k, g in grouped:
    g.to_excel(writer, k)
writer.save()

## Step 7: Create Required Dataset

For training our model/algorithm to recommend courses for students, we need a list of courses opted by each student in each semester.

In [29]:
student_sem_group = df['Course Code'].groupby([df['Student-Batch'], df['Batch / Term Code']])
student_sem_group.groups

{('Student 1 Batch 1',
  'Semester 1'): Int64Index([10, 21, 30, 33], dtype='int64'),
 ('Student 1 Batch 1',
  'Semester 2'): Int64Index([6, 22, 24, 29, 38], dtype='int64'),
 ('Student 1 Batch 1',
  'Semester 3'): Int64Index([1, 16, 18, 28, 40], dtype='int64'),
 ('Student 1 Batch 1',
  'Semester 4'): Int64Index([4, 8, 12, 25, 35], dtype='int64'),
 ('Student 1 Batch 1',
  'Semester 5'): Int64Index([0, 3, 5, 23, 37, 39], dtype='int64'),
 ('Student 1 Batch 1',
  'Semester 6'): Int64Index([2, 7, 9, 31, 32], dtype='int64'),
 ('Student 1 Batch 1',
  'Semester 7'): Int64Index([13, 14, 15, 26, 34, 36], dtype='int64'),
 ('Student 1 Batch 1',
  'Semester 8'): Int64Index([11, 17, 19, 20, 27], dtype='int64'),
 ('Student 1 Batch 2',
  'Semester 1'): Int64Index([2151, 2152, 2153, 2157], dtype='int64'),
 ('Student 1 Batch 2',
  'Semester 2'): Int64Index([2154, 2155, 2156, 2158, 2159], dtype='int64'),
 ('Student 1 Batch 3',
  'Semester 1'): Int64Index([4302, 4304, 4305, 4328, 4330], dtype='int64'),
 ('

In [30]:
dataset = []
for name, g in student_sem_group:
    courses = []
    for c in g:
        courses.append(c)
    dataset.append(courses)
opted_df = pd.DataFrame(dataset)

In [111]:
# dataset = []
# for name, g in student_sem_group:
#     courses = []
#     for c in g[g.columns[2]]:
#         courses.append(c)
#     dataset.append(courses)
# pd.DataFrame(dataset)

Save the dataset created into an excel sheet.

In [31]:
opted_df.to_excel(NEW_DATA_PATH + 'CourseSelection_Dataset.xlsx')